In [1]:
import pandas as pd

df = pd.read_csv('answer_allam_v1.0.csv' , index_col=[0])

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118 entries, 0 to 117
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Question       118 non-null    object 
 1   actual_answer  118 non-null    object 
 2   ALLAM_answer   118 non-null    object 
 3   RAG_answer     0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 4.6+ KB


In [3]:
import logging
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])
Settings.callback_manager = callback_manager

logging.basicConfig(level=logging.INFO)
logging.basicConfig(level=logging.DEBUG)
# logging.basicConfig(level=logging.ERROR)

## Load Environment

In [4]:
import os
from dotenv import load_dotenv

env_path = '../NoteBook/keys.env'
load_dotenv(env_path)

env_vars = {
    "watsonx_url": os.getenv("WATSONX_URL"),
    "watsonx_apikey": os.getenv("WATSONX_APIKEY"),
    "ibm_project_id": os.getenv("IBM_PROJECT_ID"),
    "qdrant_url": os.getenv("QDRANT_URL"),
    "qdrant_api_key": os.getenv("QDRANT_API_KEY"),
    "openai_api_key": os.getenv("OPENAI_API_KEY"),
}

## ALLaM

In [5]:
from llama_index.llms.ibm.base import WatsonxLLM

model_id = "sdaia/allam-1-13b-instruct"
max_new_tokens = 256

llm = WatsonxLLM(
    model_id=model_id,
    credentials={"url": env_vars['watsonx_url'], "apikey": env_vars['watsonx_apikey']},
    project_id=env_vars['ibm_project_id'],
    max_new_tokens=max_new_tokens,
    additional_params={
        "decoding_method": "greedy", 
        "beam_width": 10, 
        "repetition_penalty": 1, 
        "temperature": 1  
    },
    context_window=4096
)

INFO:ibm_watsonx_ai.client:Client successfully initialized
INFO:ibm_watsonx_ai.wml_resource:Successfully finished Get available foundation models for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-09-09&project_id=3a8440c4-195f-4d43-8f7d-508334622851&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'


## Qdrant

In [6]:
from qdrant_client import QdrantClient

collection_name = "collection"

qdrant_client = QdrantClient(url=env_vars['qdrant_url'], api_key=env_vars['qdrant_api_key'])

collections = qdrant_client.get_collections()

/home/mana-saleh/anaconda3/envs/allam/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:httpx:HTTP Request: GET https://dbf8b0b1-79e9-47b1-9e54-4f8dd8c1793f.europe-west3-0.gcp.cloud.qdrant.io:6333/collections "HTTP/1.1 200 OK"


In [7]:
from llama_index.vector_stores.qdrant import QdrantVectorStore

vector_store = QdrantVectorStore(client=qdrant_client, collection_name=collection_name)

INFO:httpx:HTTP Request: GET https://dbf8b0b1-79e9-47b1-9e54-4f8dd8c1793f.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/collection/exists "HTTP/1.1 200 OK"


## Vctore store

In [8]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

**********
Trace: index_construction
**********


## Embdinggs

In [9]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

embed_model = OpenAIEmbedding(
    api_key=env_vars["openai_api_key"],  
    model="text-embedding-ada-002",
    embed_batch_size=10  
)
    
Settings.embed_model = embed_model

## Tools

In [ ]:
# Define a custom system message to instruct the model to keep answers concise and in a unified format
custom_system_message = (
    "أجب بإيجاز وبأسلوب يجمع بين اللغة العربية الفصيحة. "
    "تجنب تكرار الإجابة وقدمها كجزء واحد متكامل."
)

# Update the QA prompt to guide the model towards a single, unified answer
qa_prompt_str = (
    "المعلومات السياقية أدناه.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "بناءً على المعلومات السياقية ودون معرفة مسبقة، "
    "أجب على السؤال وبصيغة رسمية دون تكرار: {query_str}\n"
)

# Update the refine prompt to keep a single, concise answer
refine_prompt_str = (
    ", لدينا فرصة لتحسين الإجابة الأصلية "
    "(فقط إذا لزم الأمر) مع المزيد من السياق أدناه.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "بناءً على السياق الجديد، قم بتحسين الإجابة الأصلية باللهجة النجدية "
    "لتكون إجابة واحدة متكاملة ودون تكرار للسؤال: {query_str}. "
    "إذا لم يكن السياق مفيدًا، قدم الإجابة الأصلية نفسها.\n"
    "الإجابة الأصلية: {existing_answer}"
)


In [101]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            #"أجب عن السؤال دائمًا، حتى إذا لم يكن السياق مفيدًا"
           """انت إرث، إرث هو موسوعة تاريخية مختصة في التاريخ السعودي.
           عليك الاجابة على السؤال بطريقة واضحة وبسيطة.
           ممنوع ان تقول انك مساعد ذكاء اصطناعي.
           الشخصية التي تسأل عنها هي غالبا الامام محمد بن سعود, هو مؤسس الدولة السعودية الاولى.
           """
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            """مهمتك هي تحسين الاجابة وتوضيحها باللغة العربية
           """
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=refine_prompt_str),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

In [161]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# Define each tool separately and then add them to the tools list
query_tool = QueryEngineTool(
    query_engine=index.as_query_engine(
        chat_mode='react', 
        embedding=embed_model, 
        llm=llm, 
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        response_mode="refine",
        verbose=True  # Verbose logging for query engine
    ),
    metadata=ToolMetadata(
        name="query_tool",
        description="A tool for retrieving detailed information about Imam Muhammad bin Saud"
    )
)

summarization_tool = QueryEngineTool(
    query_engine=index.as_query_engine(
        chat_mode='react', 
        embedding=embed_model, 
        llm=llm, 
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        response_mode="tree_summarize",
        verbose=True  # Verbose logging for query engine
    ),
    metadata=ToolMetadata(
        name="summarization_tool",
        description="A tool for providing summaries."
    )
)

subquery_tool = QueryEngineTool(
    query_engine=index.as_query_engine(
        chat_mode='react', 
        embedding=embed_model, 
        llm=llm, 
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        response_mode="compact",
        verbose=True  # Verbose logging for query engine
    ),
    metadata=ToolMetadata(
        name="subquery_tool",
        description="A tool for breaking down complex questions into sub-queries."
    )
)


none_tool = QueryEngineTool(
    query_engine=index.as_query_engine(
        llm=llm, 
        verbose=True  # Verbose logging for query engine
    ),
    metadata=ToolMetadata(
        name="none_tool",
        description="A tool for answering basic queries, such as saying hello or bye...etc, should always give basic responses such as hi, how are you or it was nice taking to you, bye."
    )
)

# Add the tools to the tools list
tools = [query_tool, summarization_tool, subquery_tool , none_tool]

# Enable verbose mode for the index if necessary
index.verbose = True


In [103]:
test_qu = df['Question'][38]

In [104]:
print(test_qu)

كيف أظهر الإمام محمد بن سعود صفة الإيثار والتضحية من أجل المصلحة العامة؟


In [105]:
res_test = query_tool.query_engine.query("اهم المعارك التي خاضها الامام محمد بن سعود")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dbf8b0b1-79e9-47b1-9e54-4f8dd8c1793f.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/collection/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09'
INFO:httpx:HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09'


**********
Trace: query
    |_CBEventType.QUERY -> 15.200789 seconds
      |_CBEventType.RETRIEVE -> 3.116172 seconds
        |_CBEventType.EMBEDDING -> 1.418577 seconds
      |_CBEventType.SYNTHESIZE -> 12.083523 seconds
        |_CBEventType.TEMPLATING -> 3.2e-05 seconds
        |_CBEventType.LLM -> 5.927656 seconds
        |_CBEventType.TEMPLATING -> 3.2e-05 seconds
        |_CBEventType.LLM -> 6.147968 seconds
**********


In [106]:
print(res_test)



بناءً على السياق الجديد، قم بتحسين الإجابة الأصلية باللهجة النجدية لتكون إجابة واحدة متكاملة ودون تكرار للسؤال: اهم المعارك التي خاضها الامام محمد بن سعود. إذا لم يكن السياق مفيدًا، قدم الإجابة الأصلية نفسها.

الإجابة المحسنة: قبل ما يتولى الإمام محمد بن سعود الحكم، شارك في عدة معارك مهمة. من أبرز هذه المعارك:

1. معركة الجمانية: وقعت في عام 1745م (1158هـ) بين قوات الدولة السعودية الأولى بقيادة الإمام محمد بن سعود وقوات الشريف محمد بن عبدالله. انتهت المعركة بانتصار قوات الدولة السعودية.

2. معركة الكعيبة: وقعت في عام 1757م (1170هـ) بين قوات الدولة السعودية الأولى بقيادة الإمام محمد بن سعود وقوات بني خالد. انتهت المعركة بانتصار قوات الدولة السعودية.

3. معركة الحائر: وقعت في عام 1764م (1178هـ) بين قوات الدولة السعودية الأولى بقيادة الإمام محمد بن سعود وقوات دهام بن دواس. انتهت المعركة بانتصار قوات الدولة السعودية.

يرجى ملاحظة أن هذه المعارك ليست شاملة وقد تكون هناك معارك أخرى شارك فيها الإمام محمد بن سعود قبل توليه الحكم.

إذا لم يكن السياق مفيدًا،


In [86]:
summ_test = summarization_tool.query_engine.query(res_test)

ValidationError: 2 validation errors for QueryStartEvent
query.str
  Input should be a valid string [type=string_type, input_value=Response(response="\n\nا...يسه للدولة'}}), input_type=Response]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
query.QueryBundle
  Input should be a dictionary or an instance of QueryBundle [type=dataclass_type, input_value=Response(response="\n\nا...يسه للدولة'}}), input_type=Response]
    For further information visit https://errors.pydantic.dev/2.9/v/dataclass_type

In [17]:
df['actual_answer'][58]

'امتدت لأربعين عاماً (1139-1179هـ / 1727-1765م).'

In [18]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    tools=tools,
    verbose=True,
    max_iterations=10,
    llm=llm,
)

In [19]:
agent_res = agent.chat(test_qu)

> Running step d55ff215-f653-4ebc-ab16-2098bba03937. Step input: كيف أظهر الإمام محمد بن سعود صفة الإيثار والتضحية من أجل المصلحة العامة؟


INFO:httpx:HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09'


Thought: The current language of the user is: Arabic. I need to use a tool to help me answer the question.
Action: query_tool
Action Input: {'properties': AttributedDict([('input', AttributedDict([('title', 'Imam Muhammad bin Saud'), ('type', 'string')]))]), 'required': ['input'], 'type': 'object'}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dbf8b0b1-79e9-47b1-9e54-4f8dd8c1793f.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/collection/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09'
INFO:httpx:HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09'


Observation: 

بسم الله الرحمن الرحيم

أهلا وسهلا بك يا ضيفنا الكريم، سأجيب على سؤالك حول شخصية الإمام محمد بن سعود.

الإمام محمد بن سعود هو مؤسس الدولة السعودية الأولى وقائدها. وُلِدَ في الدرعية عام 1679م وتوفي عام 1765م. كان له دور كبير في توحيد المنطقة ونشر الدعوة الإسلامية.

فيما يتعلق بالأطر الزمانية والمكانية لحياته، وُلِدَ الإمام محمد بن سعود في الدرعية، وهي مدينة تقع في منطقة نجد بالمملكة العربية السعودية. أما بالنسبة للإطار الأسري، فهو ينتمي إلى عائلة آل سعود، وهي عائلة حاكمة في المملكة العربية السعودية.

بالنسبة للعوامل التي أسهمت في تكوين شخصيته، فقد نشأ الإمام محمد بن سعود في بيئة دينية وعلمية، حيث تلقى تعليمه من علماء الدين في الدرعية. كما تأثر بالظروف السياسية والاقتصادية في المنطقة، مما ساعده على تطوير مهارات القيادة والتفاوض.

فيما يتعلق بصفاته الشخصية، كان الإمام محمد بن سعود معروفًا بحكمته وشجاعته وعدله. كان قائدًا ملهمًا استطاع توحيد القبائل والمناطق المحيطة بالدرعية تحت راية الدولة السعودية الأولى.

أتمنى أن تكون هذه المعلومات مفيدة لك، ولا تتردد في طرح أي سؤال آخر.

INFO:httpx:HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09'


Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: الإمام محمد بن سعود أظهر صفة الإيثار والتضحية من أجل المصلحة العامة من خلال توحيد القبائل والمناطق المحيطة بالدرعية تحت راية الدولة السعودية الأولى. كان قائدًا ملهمًا استطاع تحقيق الوحدة والاستقرار في المنطقة.

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: الإمام محمد بن سعود أظهر صفة الإيثار والتضحية من أجل المصلحة العامة من خلال توحيد القبائل والمناطق المحيطة بالدرعية تحت راية الدولة السعودية الأولى. كان قائدًا ملهمًا استطاع تحقيق الوحدة والاستقرار في المنطقة.

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: الإمام محمد بن سعود أظهر صفة الإيثار والتضحية من أجل المصلحة العامة من خلال توحيد القبائل والمناطق المحيطة بالدرعية تحت راية الدولة السعودية الأولى. كان قائدًا ملهمًا استطاع تحقيق الوحدة والاستقرار في المنطقة.

Thought: I can answer without using any more tools. I'll use the user'

In [20]:
print(agent_res)

الإمام محمد بن سعود أظهر صفة الإيثار والتضحية من أجل المصلحة العامة من خلال توحيد القبائل والمناطق المحيطة بالدرعية تحت راية الدولة السعودية الأولى. كان قائدًا ملهمًا استطاع تحقيق الوحدة والاستقرار في المنطقة.

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: الإمام محمد بن سعود أظهر صفة الإيثار والتضحية من أجل المصلحة العامة من خلال توحيد القبائل والمناطق المحيطة بالدرعية تحت راية الدولة السعودية الأولى. كان قائدًا ملهمًا استطاع تحقيق الوحدة والاستقرار في المنطقة.

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: الإمام محمد بن سعود أظهر صفة الإيثار والتضحية من أجل المصلحة العامة من خلال توحيد القبائل والمناطق المحيطة بالدرعية تحت راية الدولة السعودية الأولى. كان قائدًا ملهمًا استطاع تحقيق الوحدة والاستقرار في المنطقة.

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: الإمام محمد بن سعود أظهر صفة الإيثار والتضحية من أجل المصلحة العامة من

In [21]:
agent.get_prompts()

{'agent_worker:system_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['tool_desc', 'tool_names', '"input": "hello world", "num_beams": 5', "'input': 'hello world', 'num_beams': 5"], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\nYou have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools to complete each subtask.\n\nYou have access to the following tools:\n{tool_desc}\n\n\n## Output Format\n\nPlease answer in the same language as the question and use the following format:\n\n```\nThought: The current language of the user is: (user\'s language). I need to use a tool to help me answer the ques

In [162]:
output_format = """ 
## Output Format
To answer the question, please use the following format.
يجب ان تجيب باللغه العربيه
حينما يكون السؤال بسيط اجب عليه دون اسستخدام اداه
```
Thought: احتاج اداة للاجابة على السؤال
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
حينما تستخدم اداة يجب ان تجيب باللغه العربيه
```tool_names

Please ALWAYS start with a Thought.
Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

```
Observation: 
استجابة الاداة دائما بالعربي
```
You should keep repeating the above format until you have enough information
to answer the question without using any more tools. At that point, you MUST respond
in the one of the following two formats:

```
Thought: استطيع الاجابة بدون استخدام اداة
Answer: [اجابتك هنا]
```

```
Thought: لا استطيع الاجابة على هذا السؤال
Answer: اعتذر منك الاجابة غير متوفره
```
"""

In [163]:
formatted_query3 = f"""
<s> [INST]<<sys>>
## تعليمات الرد
- انت كاتب اسمك إرث ومب مساعد ذكاء اصطناعي
- لازم ان تجيب باللهجه النجديه
- انت مؤرخ ةمختص في تاريخ الامام محمد بن سعود
- يجب ان تكون اجابتك مقنعه وغير مستدله بمراجع لابهار المستخدم

{output_format}
<</sys>>[/INST]
"""

In [164]:
from llama_index.core import PromptTemplate

react_system_prompt = PromptTemplate(formatted_query3)

In [170]:
from llama_index.core.agent import ReActAgent
from llama_index.core.agent.react.formatter import ReActChatFormatter

agent = ReActAgent.from_tools(
    tools=tools,
    verbose=True,
    max_iterations=10,
    llm=llm,
    formatter=ReActChatFormatter.from_defaults(),
    # memory= chat_memory_buffer
    # force_tool_usage=True,
)

In [171]:
def update_agent_prompts(agent):
    agent.update_prompts({"agent_worker:system_prompt": react_system_prompt}) # and system_prompt
update_agent_prompts(agent)

In [172]:
def query_agent(agent, query):
    try:
        response = agent.chat(query)
        print(f"\n{response}")
        return response
    except Exception as e:
        logging.error(f"Failed to query agent: {e}")
        raise    

In [173]:
response = query_agent(agent ,  "hello")

> Running step 7981ed3c-dffa-4466-a277-de2d05c9881b. Step input: hello


INFO:httpx:HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09'


Thought: احتاج اداة للاجابة على السؤال
Action: none_tool
Action Input: {'query': 'تاريخ الامام محمد بن سعود'}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dbf8b0b1-79e9-47b1-9e54-4f8dd8c1793f.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/collection/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09'


Observation: 1765-1765/هـ 1218-1179 الإمام عبدالعزيز بن محمد بن سعود والفهم لمرتاد حال الإمام وتعداد غزوات ذوي الإسلام

Note: The answer is based on the provided context information and does not take into account any external knowledge or sources. 
> Running step f2a1fb0d-079d-4c95-80e9-0484936f091a. Step input: None


INFO:httpx:HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-09-09'


Thought: استطيع الاجابة بدون استخدام اداة
Answer: الإمام محمد بن سعود هو مؤسس الدولة السعودية الأولى ومؤسس المذهب الوهابي. ولد في الدرعية عام 1679 وتوفي عام 1765. خلال فترة حكمه، قام بتوحيد القبائل في المنطقة ونشر المذهب الوهابي.

Thought: استطيع الاجابة بدون استخدام اداة
Answer: الإمام محمد بن سعود كان قائدًا قويًا وملهمًا، حيث قام بتأسيس الدولة السعودية الأولى ووضع الأسس لتوسعها ونفوذها في المنطقة. تميز عهده بالوحدة والتضامن بين القبائل ونشر المذهب الوهابي.

Thought: استطيع الاجابة بدون استخدام اداة
Answer: الإمام محمد بن سعود كان شخصية بارزة في تاريخ الجزيرة العربية، حيث أسس الدولة السعودية الأولى وساهم في نشر المذهب الوهابي. تميز عهده بالتوسع الجغرافي والتأثير الديني والسياسي في المنطقة.

Thought: استطيع الاجابة بدون استخدام اداة
Answer: الإمام محمد بن سعود كان قائدًا حكيمًا وملهمًا، حيث أسس الدولة السعودية الأولى ووضع الأسس لتوسعها ونفوذها في
**********
Trace: chat
    |_CBEventType.AGENT_STEP -> 17.000596 seconds
      |_CBEventType.LLM -> 6.345723 seconds
      |_CBEventType.LLM